## Imports

In [1]:
import datetime
import gc

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *

In [3]:
cuda_use_gpus(1)

In [4]:
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


## Config

In [5]:
feature_list_id = 'oofp_bradleypallen_mlp_magic'

In [6]:
RANDOM_SEED = 42

In [7]:
np.random.seed(RANDOM_SEED)

## Read Data

In [8]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [9]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [10]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [11]:
magic_feature_lists = [
    'magic_jturkewitz',
    'magic_stas_svd_150',
    'magic_stas_avito',
]

In [12]:
X_train_magic, X_test_magic, _ = load_feature_lists(magic_feature_lists)

In [13]:
X_train_magic = X_train_magic.values
X_test_magic = X_test_magic.values

In [14]:
scaler = StandardScaler()
scaler.fit(np.vstack([X_train_magic, X_test_magic]))
X_train_magic = scaler.transform(X_train_magic)
X_test_magic = scaler.transform(X_test_magic)

In [15]:
y_train = load(features_data_folder + 'y_train.pickle')

In [16]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0]
MAX_SEQUENCE_LENGTH = X_train_q1.shape[-1]

In [17]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

300 101442 30


## Train Models & Compute Out-of-Fold Predictions

In [18]:
def create_model_question_branch():
    input_q = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    
    embedding_q = Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    )(input_q)

    timedist_q = TimeDistributed(Dense(
        EMBEDDING_DIM,
        activation='relu',
    ))(embedding_q)

    lambda_q = Lambda(
        lambda x: K.max(x, axis=1),
        output_shape=(EMBEDDING_DIM, )
    )(timedist_q)
    
    output_q = lambda_q
    return input_q, output_q

In [19]:
def create_model(params):
    q1_input, q1_output = create_model_question_branch()
    q2_input, q2_output = create_model_question_branch()
    magic_input = Input(shape=(X_train_magic.shape[-1], ))
    
    merged_inputs = concatenate([q1_output, q2_output, magic_input])

    dense_1 = Dense(params['num_dense_1'])(merged_inputs)
    bn_1 = BatchNormalization()(dense_1)
    relu_1 = Activation('relu')(bn_1)

    dense_2 = Dense(params['num_dense_2'])(relu_1)
    bn_2 = BatchNormalization()(dense_2)
    relu_2 = Activation('relu')(bn_2)

    dense_3 = Dense(params['num_dense_3'])(relu_2)
    bn_3 = BatchNormalization()(dense_3)
    relu_3 = Activation('relu')(bn_3)

    dense_4 = Dense(params['num_dense_4'])(relu_3)
    bn_4 = BatchNormalization()(dense_4)
    relu_4 = Activation('relu')(bn_4)

    bn_final = BatchNormalization()(relu_4)
    output = Dense(1, activation='sigmoid')(bn_final)
    
    model = Model(
        inputs=[q1_input, q2_input, magic_input],
        outputs=output,
    )

    model.compile(
        loss='binary_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
    )

    return model

In [20]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [21]:
NUM_FOLDS = 5
NUM_EPOCHS = 200

In [22]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [23]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [24]:
y_test_oofp = np.zeros((len(X_test_q1), NUM_FOLDS))

In [25]:
model_params = {
    'num_dense_1': 400,
    'num_dense_2': 200,
    'num_dense_3': 400,
    'num_dense_4': 100,
}

In [26]:
def predict(model, X_q1, X_q2, X_magic):
    y1 = model.predict(
        [X_q1, X_q2, X_magic],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    y2 = model.predict(
        [X_q2, X_q1, X_magic],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    return (y1 + y2) / 2

In [27]:
%%time

for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):
    X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
    X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])
    X_fold_train_magic = np.vstack([X_train_magic[ix_train], X_train_magic[ix_train]])

    X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
    X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])
    X_fold_val_magic = np.vstack([X_train_magic[ix_val], X_train_magic[ix_val]])

    y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
    y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model(model_params)
    model.fit(
        [X_fold_train_q1, X_fold_train_q2, X_fold_train_magic], y_fold_train,
        validation_data=([X_fold_val_q1, X_fold_val_q2, X_fold_val_magic], y_fold_val),

        batch_size=64,
        epochs=NUM_EPOCHS,
        verbose=1,
        
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_train_oofp[ix_val] = predict(model, X_train_q1[ix_val], X_train_q2[ix_val], X_train_magic[ix_val])
    y_test_oofp[:, fold_num] = predict(model, X_test_q1, X_test_q2, X_test_magic)
    
    # Clear GPU memory.
    K.clear_session()
    del X_fold_train_q1
    del X_fold_train_q2
    del X_fold_train_magic
    del X_fold_val_q1
    del X_fold_val_q2
    del X_fold_val_magic
    del model
    gc.collect()


Fitting fold 1 of 5

Train on 646862 samples, validate on 161718 samples
Epoch 1/200
646862/646862 [==============================] - 68s - loss: 0.3364 - acc: 0.8501 - val_loss: 0.3082 - val_acc: 0.8600
Epoch 2/200
646862/646862 [==============================] - 68s - loss: 0.2847 - acc: 0.8724 - val_loss: 0.2959 - val_acc: 0.8693
Epoch 3/200
646862/646862 [==============================] - 68s - loss: 0.2554 - acc: 0.8867 - val_loss: 0.2827 - val_acc: 0.8749
Epoch 4/200
646862/646862 [==============================] - 67s - loss: 0.2320 - acc: 0.8976 - val_loss: 0.3019 - val_acc: 0.8626
Epoch 5/200
646862/646862 [==============================] - 66s - loss: 0.2111 - acc: 0.9080 - val_loss: 0.2882 - val_acc: 0.8719
Epoch 6/200
646862/646862 [==============================] - 62s - loss: 0.1930 - acc: 0.9163 - val_loss: 0.2888 - val_acc: 0.8744
Epoch 7/200
646862/646862 [==============================] - 62s - loss: 0.1757 - acc: 0.9242 - val_loss: 0.3136 - val_acc: 0.8629
Epoch 000

646866/646866 [==============================] - 69s - loss: 0.3369 - acc: 0.8504 - val_loss: 0.3035 - val_acc: 0.8637
Epoch 2/200
646866/646866 [==============================] - 68s - loss: 0.2857 - acc: 0.8729 - val_loss: 0.2909 - val_acc: 0.8702
Epoch 3/200
646866/646866 [==============================] - 68s - loss: 0.2568 - acc: 0.8865 - val_loss: 0.2809 - val_acc: 0.8750
Epoch 4/200
646866/646866 [==============================] - 68s - loss: 0.2332 - acc: 0.8978 - val_loss: 0.2864 - val_acc: 0.8772
Epoch 5/200
646866/646866 [==============================] - 68s - loss: 0.2128 - acc: 0.9072 - val_loss: 0.2871 - val_acc: 0.8785
Epoch 6/200
646866/646866 [==============================] - 68s - loss: 0.1941 - acc: 0.9164 - val_loss: 0.2787 - val_acc: 0.8811
Epoch 7/200
646866/646866 [==============================] - 68s - loss: 0.1779 - acc: 0.9236 - val_loss: 0.2898 - val_acc: 0.8781
Epoch 8/200
646866/646866 [==============================] - 68s - loss: 0.1627 - acc: 0.9303 -

In [28]:
cv_score = log_loss(y_train, y_train_oofp)
print('CV score:', cv_score)

CV score: 0.27110014293


## Save feature names

In [29]:
feature_names = [
    'oofp_bradleypallen_mlp_magic',
]

In [30]:
save_lines(feature_names, features_data_folder + f'X_train_{feature_list_id}.names')

## Save Train features

In [31]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [32]:
save(y_train_oofp, features_data_folder + f'X_train_{feature_list_id}.pickle')

## Save Test features

In [33]:
y_test_oofp_mean = np.mean(y_test_oofp, axis=1).reshape((-1, 1))

In [34]:
save(y_test_oofp_mean, features_data_folder + f'X_test_{feature_list_id}.pickle')